In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import json
import folium
warnings.filterwarnings("ignore")


In [2]:
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

In [3]:
traffic_Seoul = pd.read_csv('../Data/newSeoul_2005_2019.csv')
traffic_Seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [5]:
# 2019년 데이터만 추출하기
traffic_Seoul_2019 = traffic_Seoul[traffic_Seoul['년도']==2019]
traffic_Seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
4200,2019,1,종로구,87,1,125
4201,2019,2,종로구,66,1,84
4202,2019,3,종로구,87,2,122
4203,2019,4,종로구,85,0,131
4204,2019,5,종로구,112,1,158


In [6]:
# index 정리하기
traffic_Seoul_2019.reset_index(drop=True, inplace=True)
traffic_Seoul_2019

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158
...,...,...,...,...,...,...
295,2019,8,강동구,127,2,175
296,2019,9,강동구,98,2,137
297,2019,10,강동구,108,1,144
298,2019,11,강동구,146,0,199


In [7]:
# 자치구별 발생건수, 사망자수, 부상자수 집계하기
traffic_anal = pd.pivot_table(traffic_Seoul_2019, index = '자치구명', aggfunc = sum)
traffic_anal.head()

,년도,발생건수,부상자수,사망자수,월
자치구명,,,,,
강남구,24228,3722,5182,14,78
강동구,24228,1414,1910,11,78
강북구,24228,1277,1706,7,78
강서구,24228,1829,2491,20,78
관악구,24228,1363,1755,10,78


In [8]:
# 년도와 월은 제거하여 Dataset 만들기
traffic_anal = traffic_anal[['발생건수','부상자수','사망자수']]
traffic_anal.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [11]:
seoul_limit = pd.read_csv('../Data/seoul.csv', encoding='euc-kr')#encoding='utf-8')
seoul_limit.columns=['자치구명','lon','lat']
seoul_limit.head()


,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [13]:
data_result.isnull().any()

자치구명    False
발생건수    False
부상자수    False
사망자수    False
lon     False
lat     False
dtype: bool

In [14]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ KB


In [15]:
# folium을 사용하기 위해서 int를 float로 변환
data_result['발생건수'] = data_result['발생건수'].astype(float)
data_result['부상자수'] = data_result['부상자수'].astype(float)
data_result['사망자수'] = data_result['사망자수'].astype(float)

In [16]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.4+ KB


In [24]:
# Map 만들기
map = folium.Map(
    location = [37.55, 126.98], 
    tiles = 'Stamen Terrain', 
    # tiles = 'Stamen Toner', 
    zoom_start = 12
    )
for name, lat, lng in zip(data_result.자치구명, data_result.lat, data_result.lon):
    try:
        folium.CircleMarker([lat, lng],
                            radius = 10,
                            color = 'brown',
                            fill = True,
                            fill_color = 'coral',
                            fill_opacity = 0.7,
                            popup = name).add_to(map)
    except:print(f'{name} error')
    


map

In [23]:
# 강사님 답안

map = folium.Map(
    location = [37.55, 126.98], 
    zoom_start = 11
    )
for n in data_result.index:
    folium.Marker(
        [data_result['lat'][n], data_result['lon'][n]],
        popup=data_result['자치구명'][n]
    ).add_to(map)

map

In [35]:
# 강사님 답안

map = folium.Map(
    location = [37.55, 126.98], 
    zoom_start = 11
    )
for n in data_result.index:
    folium.Marker(
        [data_result['lat'][n], data_result['lon'][n]],
        popup=data_result['자치구명'][n]
    ).add_to(map)
    folium.CircleMarker(
        [data_result['lat'][n], data_result['lon'][n]],
        radius=data_result['발생건수'][n] / 100,
        color = 'blue',
        fill_color='sky blue',
        fill=True
        ).add_to(map)
    folium.CircleMarker(
        [data_result['lat'][n], data_result['lon'][n]],
        radius=data_result['사망자수'][n],
        color = 'red',
        fill_color='red',
        fill=True
        ).add_to(map)

map